# Description

This notebook is used to scrape Fbref for the Scouting Dashboard submitted for the Football Insights Analyst position @ CityFootballGroup

# Setup

In [2]:
#Data management
import pandas as pd
import numpy as np

#File management
import os
import os.path as osp

#Scraping
from ScraperFC import FBRef
from ScraperFC.shared_functions import sources

c:\Users\edoar\anaconda3\envs\football\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Scraping

In [2]:
sources['FBRef']['Big 5 combined']

{'first valid year': 1996,
 'url': 'https://fbref.com/en/comps/Big5/history/Big-5-European-Leagues-Seasons',
 'finder': ['Big-5-European-Leagues']}

## Stats

In [8]:
scraper = FBRef()
stats_categories = scraper.stats_categories

In [9]:
stats_categories

{'standard': {'url': 'stats', 'html': 'standard'},
 'goalkeeping': {'url': 'keepers', 'html': 'keeper'},
 'advanced goalkeeping': {'url': 'keepersadv', 'html': 'keeper_adv'},
 'shooting': {'url': 'shooting', 'html': 'shooting'},
 'passing': {'url': 'passing', 'html': 'passing'},
 'pass types': {'url': 'passing_types', 'html': 'passing_types'},
 'goal and shot creation': {'url': 'gca', 'html': 'gca'},
 'defensive': {'url': 'defense', 'html': 'defense'},
 'possession': {'url': 'possession', 'html': 'possession'},
 'playing time': {'url': 'playingtime', 'html': 'playing_time'},
 'misc': {'url': 'misc', 'html': 'misc'}}

In [5]:
year = 2023
league = 'Big 5 combined'

In [6]:
stats_df = {k:scraper.scrape_stats(year=year, league=league, stat_category=k) for k in stats_categories.keys()}

In [10]:
players_stats_dfs = {k:v[2].copy() for k,v in stats_df.items()}

In [11]:
players_stats_dfs.keys()

dict_keys(['standard', 'goalkeeping', 'advanced goalkeeping', 'shooting', 'passing', 'pass types', 'goal and shot creation', 'defensive', 'possession', 'playing time', 'misc'])

# Data Manipulation

In [37]:
initial_cols = {k:list(v.columns)[:7] for k,v in players_stats_dfs.items()}

In [39]:
for v in initial_cols.values():
    print(v)

[('Unnamed: 0_level_0', 'Rk'), ('Unnamed: 1_level_0', 'Player'), ('Unnamed: 2_level_0', 'Nation'), ('Unnamed: 3_level_0', 'Pos'), ('Unnamed: 4_level_0', 'Squad'), ('Unnamed: 5_level_0', 'Comp'), ('Unnamed: 6_level_0', 'Age')]
[('Unnamed: 0_level_0', 'Rk'), ('Unnamed: 1_level_0', 'Player'), ('Unnamed: 2_level_0', 'Nation'), ('Unnamed: 3_level_0', 'Pos'), ('Unnamed: 4_level_0', 'Squad'), ('Unnamed: 5_level_0', 'Comp'), ('Unnamed: 6_level_0', 'Age')]
[('Unnamed: 0_level_0', 'Rk'), ('Unnamed: 1_level_0', 'Player'), ('Unnamed: 2_level_0', 'Nation'), ('Unnamed: 3_level_0', 'Pos'), ('Unnamed: 4_level_0', 'Squad'), ('Unnamed: 5_level_0', 'Comp'), ('Unnamed: 6_level_0', 'Age')]
[('Unnamed: 0_level_0', 'Rk'), ('Unnamed: 1_level_0', 'Player'), ('Unnamed: 2_level_0', 'Nation'), ('Unnamed: 3_level_0', 'Pos'), ('Unnamed: 4_level_0', 'Squad'), ('Unnamed: 5_level_0', 'Comp'), ('Unnamed: 6_level_0', 'Age')]
[('Unnamed: 0_level_0', 'Rk'), ('Unnamed: 1_level_0', 'Player'), ('Unnamed: 2_level_0', 'Nation'

### Anagrafical Data

In [92]:
anagrafical_data_file = 'anagrafical_data.csv'
file_path = osp.join('data',anagrafical_data_file)


anagrafical_df = players_stats_dfs['standard'].copy()
anagrafical_df.columns = anagrafical_df.columns.droplevel()
anagrafical_df = anagrafical_df.loc[:, :'Age']

In [93]:
anagrafical_df.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age
0,1,Brenden Aaronson,us USA,"MF,FW",Leeds United,eng Premier League,21
1,2,Paxten Aaronson,us USA,"MF,DF",Eint Frankfurt,de Bundesliga,18
2,3,James Abankwah,ie IRL,DF,Udinese,it Serie A,18
3,4,George Abbott,eng ENG,MF,Tottenham,eng Premier League,16
4,5,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,34


In [94]:
#Nation
anagrafical_df['Nation'] = anagrafical_df['Nation'].apply(lambda x: x.split()[1] if x is not np.nan else x)

In [95]:
#Main position
anagrafical_df['Main_position'] = anagrafical_df['Pos'].apply(lambda x : x.split(',')[0] if len(x.split(','))>1 else x)

In [96]:
#Second_position
anagrafical_df['Second_position'] = anagrafical_df['Pos'].apply(lambda x : x.split(',')[1] if len(x.split(','))>1 else np.nan)

In [97]:
#Comp
anagrafical_df['Comp'] = anagrafical_df['Comp'].apply(lambda x : ' '.join(x.split()[1:]))

In [98]:
#Nation competition
comps = anagrafical_df['Comp'].unique()
nation_comp = {
    'Premier League' : 'United Kingdom',
    'Serie A' : 'Italy',
    'Ligue 1' : 'France',
    'La Liga' : 'Spain',
    'Bundesliga' : 'Germany'
}

anagrafical_df['Nation_comp'] = anagrafical_df['Comp'].map(nation_comp)

In [99]:
#Player Id
anagrafical_df['Player_id'] = players_stats_dfs['standard']['Player ID']

In [100]:
anagrafical_df = anagrafical_df.drop(columns=['Rk'])

In [102]:
anagrafical_df.shape, anagrafical_df['Player_id'].duplicated().sum()

((2889, 10), 164)

In [103]:
#Remove the double rows for those player that are moved during transfer window
#The current team is in the first player occurance 
anagrafical_df = anagrafical_df[~anagrafical_df['Player_id'].duplicated()]
anagrafical_df

,Player,Nation,Pos,Squad,Comp,Age,Main_position,Second_position,Nation_comp,Player_id
0,Brenden Aaronson,USA,"MF,FW",Leeds United,Premier League,21,MF,FW,United Kingdom,5bc43860
1,Paxten Aaronson,USA,"MF,DF",Eint Frankfurt,Bundesliga,18,MF,DF,Germany,4cd41883
2,James Abankwah,IRL,DF,Udinese,Serie A,18,DF,NaN,Italy,4b7a9fcc
3,George Abbott,ENG,MF,Tottenham,Premier League,16,MF,NaN,United Kingdom,56628958
4,Yunis Abdelhamid,MAR,DF,Reims,Ligue 1,34,DF,NaN,France,32c2d95f
...,...,...,...,...,...,...,...,...,...,...
2884,Martin Ødegaard,NOR,MF,Arsenal,Premier League,23,MF,NaN,United Kingdom,79300479
2885,Milan Đurić,BIH,FW,Hellas Verona,Serie A,32,FW,NaN,Italy,405f6586
2886,Filip Đuričić,SRB,"MF,FW",Sampdoria,Serie A,30,MF,FW,Italy,eccbe56a
2887,Blanco,NaN,MF,Cádiz,La Liga,22,MF,NaN,Spain,e8deefd6


In [104]:
anagrafical_df.to_csv(file_path, index=False)

### Stats

In [150]:
cleaned_stats = {}

In [180]:
players_stats_dfs['standard'].columns

MultiIndex([( 'Unnamed: 0_level_0',       'Rk'),
            ( 'Unnamed: 1_level_0',   'Player'),
            ( 'Unnamed: 2_level_0',   'Nation'),
            ( 'Unnamed: 3_level_0',      'Pos'),
            ( 'Unnamed: 4_level_0',    'Squad'),
            ( 'Unnamed: 5_level_0',     'Comp'),
            ( 'Unnamed: 6_level_0',      'Age'),
            ( 'Unnamed: 7_level_0',     'Born'),
            (       'Playing Time',       'MP'),
            (       'Playing Time',   'Starts'),
            (       'Playing Time',      'Min'),
            (       'Playing Time',      '90s'),
            (        'Performance',      'Gls'),
            (        'Performance',      'Ast'),
            (        'Performance',      'G+A'),
            (        'Performance',     'G-PK'),
            (        'Performance',       'PK'),
            (        'Performance',    'PKatt'),
            (        'Performance',     'CrdY'),
            (        'Performance',     'CrdR'),
            (       

In [181]:
additional_columns = [(          'Player ID',         ''),( 'Unnamed: 1_level_0',   'Player'), ]

standard_metrics = [
    (       'Playing Time',       'MP'),
    (       'Playing Time',       '90s'),
    (       'Playing Time',       'Min'),
    (       'Playing Time',       'Starts'),
    (        'Performance',      'Gls'),
    (        'Performance',     'G-PK'),
    (           'Expected',     'npxG'),
    (           'Expected',       'xG'),
    (        'Progression',     'PrgC'),
    (        'Progression',     'PrgP'),
    ]

gk_metrics = [
    (        'Performance',      'CS'),
    (        'Performance',    'SoTA'),
    (        'Performance',   'Saves'),
    (        'Performance',      'GA'),
    ]


adv_gk_metrics = [
    (           'Expected',  'PSxG+/-'),
    (            'Crosses',      'Stp'),
    (            'Crosses',      'Opp'),
]

shooting_metrics = [
    (           'Standard',     'SoT'),
]

passing_metrics = [
    (              'Total',     'Cmp'),
    (              'Total',     'Att'),
    ('Unnamed: 23_level_0',     'Ast'),
    ('Unnamed: 24_level_0',     'xAG'),
    (           'Expected',   'A-xAG'),
    ('Unnamed: 28_level_0',     '1/3'),
]

defensive_metrics = [
    (         'Challenges',     'Tkl'),
    (         'Challenges',     'Att'),
    ('Unnamed: 22_level_0', 'Tkl+Int'),
    (            'Tackles', 'Def 3rd'),
    (             'Blocks',  'Blocks'),
    ('Unnamed: 23_level_0',     'Clr'),
]


metrics = {
    'standard': standard_metrics,
    'goalkeeping': gk_metrics,
    'advanced goalkeeping': adv_gk_metrics,
    'shooting': shooting_metrics,
    'passing':passing_metrics,
    'defensive':defensive_metrics
}


for k,v in metrics.items():
    temp_df = players_stats_dfs[k].copy()
    temp_df = temp_df[additional_columns+v]
    temp_df.columns = [col[1] if col[1] != '' else col[0] for col in temp_df.columns]
    temp_df.iloc[:,2:] = temp_df.iloc[:,2:].astype(float)
    temp_df = temp_df.groupby(by=['Player ID', 'Player']).sum()
    temp_df.reset_index(inplace=True)
    print(temp_df.shape)
    cleaned_stats[k] = temp_df
    
     


(2725, 12)
(202, 6)
(202, 5)
(2725, 3)
(2725, 8)
(2725, 8)


**Renaming Columns**

In [182]:
standard_rename = {
    'Gls':'Goals',
    'G-PK':'npGoals',
    'PrgC':'PrgCarries',
    'PrgP':'PrgPasses',
    'MP':'Matches Played',
    'Min':'Minutes Played',
}

adv_gk_rename = {
    'PSxG+/-': 'PSxGA - GA',
    'Stp': 'Crosses Stp',
    'Opp': 'Crosses Opp',
}

passing_rename = {
    'Cmp': 'Passes Cmp',
    'Att': 'Passes Att',
    '1/3': 'Passes Final 3rd'
}

defensive_metrics = {
    'Tkl':'Challenges Tkl',
    'Att':'Challenges Against Att',
    'Def 3rd': 'Tackles Def 3rd', 
}

rename_dict = {
    'standard':standard_rename,
    'advanced goalkeeping': adv_gk_rename,
    'passing':passing_rename,
    'defensive':defensive_metrics
}

for k,v in rename_dict.items():
    cleaned_stats[k] = cleaned_stats[k].rename(columns=v)

### Merging

In [207]:
final_df = cleaned_stats['standard']

for cleaned_df in list(cleaned_stats.values())[1:]:
    final_df = pd.merge(final_df, cleaned_df, on=['Player ID', 'Player'], how='left')

final_df

,Player ID,Player,Matches Played,90s,Minutes Played,Starts,Goals,npGoals,npxG,xG,...,Ast,xAG,A-xAG,Passes Final 3rd,Challenges Tkl,Challenges Against Att,Tkl+Int,Tackles Def 3rd,Blocks,Clr
0,0000acda,Marco Benassi,17.0,12.1,1091.0,12.0,0.0,0.0,1.3,1.3,...,0.0,0.5,-0.5,28.0,18.0,28.0,42.0,11.0,13.0,20.0
1,00242715,Moussa Niakhate,14.0,12.9,1163.0,14.0,0.0,0.0,0.6,0.6,...,0.0,0.1,-0.1,16.0,11.0,22.0,37.0,15.0,18.0,60.0
2,00242b75,Leandro Barreiro Martins,31.0,23.8,2141.0,25.0,4.0,4.0,3.9,3.9,...,3.0,2.2,0.8,48.0,22.0,49.0,101.0,19.0,38.0,39.0
3,00459419,Marko Arnautović,21.0,17.0,1530.0,18.0,10.0,7.0,4.8,7.2,...,0.0,1.9,-1.9,17.0,1.0,6.0,11.0,2.0,11.0,10.0
4,004d185e,Tete Morente,32.0,23.0,2068.0,23.0,4.0,4.0,4.2,4.2,...,0.0,1.1,-1.1,44.0,15.0,36.0,58.0,18.0,23.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2720,ff98492b,Robert Skov,23.0,15.0,1349.0,16.0,3.0,1.0,1.8,3.4,...,2.0,1.8,0.2,35.0,10.0,26.0,30.0,14.0,22.0,20.0
2721,ffacd3d5,Manu Sánchez,31.0,23.6,2124.0,22.0,0.0,0.0,0.8,0.8,...,2.0,2.1,-0.1,72.0,33.0,63.0,65.0,35.0,33.0,51.0
2722,ffec9769,Islam Slimani,16.0,11.0,986.0,11.0,1.0,1.0,1.7,2.5,...,1.0,1.2,-0.2,22.0,0.0,3.0,6.0,2.0,2.0,11.0
2723,ffed43e3,Jack Hendry,4.0,2.4,220.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,12.0,0.0,1.0,1.0,0.0,2.0,7.0


### P90

In [208]:
p90_metrics = [
    'Goals', 'npGoals', 'npxG', 'xG','SoT',
    'PrgCarries','PrgPasses',
    'xAG'
]

def compute_p90_metric(x, metric):
    if x['90s'] > 0:
        return x[metric]/x['90s']
    else:
        return x[metric]

for p90m in p90_metrics:
    final_df[f"{p90m}_p90"] = final_df[[p90m, '90s']].apply(lambda x: compute_p90_metric(x, p90m), axis=1)

final_df

,Player ID,Player,Matches Played,90s,Minutes Played,Starts,Goals,npGoals,npxG,xG,...,Blocks,Clr,Goals_p90,npGoals_p90,npxG_p90,xG_p90,SoT_p90,PrgCarries_p90,PrgPasses_p90,xAG_p90
0,0000acda,Marco Benassi,17.0,12.1,1091.0,12.0,0.0,0.0,1.3,1.3,...,13.0,20.0,0.000000,0.000000,0.107438,0.107438,0.661157,0.826446,2.975207,0.041322
1,00242715,Moussa Niakhate,14.0,12.9,1163.0,14.0,0.0,0.0,0.6,0.6,...,18.0,60.0,0.000000,0.000000,0.046512,0.046512,0.232558,0.155039,1.162791,0.007752
2,00242b75,Leandro Barreiro Martins,31.0,23.8,2141.0,25.0,4.0,4.0,3.9,3.9,...,38.0,39.0,0.168067,0.168067,0.163866,0.163866,0.588235,0.714286,2.268908,0.092437
3,00459419,Marko Arnautović,21.0,17.0,1530.0,18.0,10.0,7.0,4.8,7.2,...,11.0,10.0,0.588235,0.411765,0.282353,0.423529,0.882353,0.764706,2.117647,0.111765
4,004d185e,Tete Morente,32.0,23.0,2068.0,23.0,4.0,4.0,4.2,4.2,...,23.0,28.0,0.173913,0.173913,0.182609,0.182609,0.608696,2.608696,2.739130,0.047826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2720,ff98492b,Robert Skov,23.0,15.0,1349.0,16.0,3.0,1.0,1.8,3.4,...,22.0,20.0,0.200000,0.066667,0.120000,0.226667,0.800000,2.866667,3.266667,0.120000
2721,ffacd3d5,Manu Sánchez,31.0,23.6,2124.0,22.0,0.0,0.0,0.8,0.8,...,33.0,51.0,0.000000,0.000000,0.033898,0.033898,0.127119,1.779661,4.237288,0.088983
2722,ffec9769,Islam Slimani,16.0,11.0,986.0,11.0,1.0,1.0,1.7,2.5,...,2.0,11.0,0.090909,0.090909,0.154545,0.227273,0.181818,1.000000,2.363636,0.109091
2723,ffed43e3,Jack Hendry,4.0,2.4,220.0,2.0,0.0,0.0,0.0,0.0,...,2.0,7.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.416667,3.750000,0.000000


### % Metrics

In [40]:
# Passes Completed -> Passes Cmp/Passes Att
# % Tkl -> 'Challenges Tkl'/ 'Challenges Against Att'
# % Saves -> 'Saves' / 'SoTA'
# % Crs Stopped -> 'Crosses Stp' / 'Crosses Opp'

import math


metrics_percentage = {
    '%PassesCmp': ['Passes Cmp','Passes Att'],
    '%TklCmp':['Challenges Tkl', 'Challenges Against Att'],
    '%Saves':[ 'Saves', 'SoTA'],
    '%CossesStopped':['Crosses Stp','Crosses Opp']
}

def compute_metric_percentage(x, prim_metrics):
    if x[prim_metrics[1]] > 0 :
        return x[prim_metrics[0]] / x[prim_metrics[1]]*100
    elif math.isnan(x[prim_metrics[1]]):
        return np.nan
    else:
        return 0

for mperc,prim_metrics in metrics_percentage.items():
    final_df[mperc] = final_df[prim_metrics].apply(lambda x : compute_metric_percentage(x,prim_metrics), axis=1)

In [42]:
for mperc in metrics_percentage:
    final_df[mperc] = final_df[mperc]*100

In [43]:
final_df

,Player ID,Player,Matches Played,90s,Minutes Played,Starts,Goals,npGoals,npxG,xG,...,npxG_p90,xG_p90,SoT_p90,PrgCarries_p90,PrgPasses_p90,xAG_p90,%PassesCmp,%TklCmp,%Saves,%CossesStopped
0,0000acda,Marco Benassi,17.0,12.1,1091.0,12.0,0.0,0.0,1.3,1.3,...,0.107438,0.107438,0.661157,0.826446,2.975207,0.041322,75.501114,64.285714,NaN,NaN
1,00242715,Moussa Niakhate,14.0,12.9,1163.0,14.0,0.0,0.0,0.6,0.6,...,0.046512,0.046512,0.232558,0.155039,1.162791,0.007752,67.774936,50.000000,NaN,NaN
2,00242b75,Leandro Barreiro Martins,31.0,23.8,2141.0,25.0,4.0,4.0,3.9,3.9,...,0.163866,0.163866,0.588235,0.714286,2.268908,0.092437,68.043088,44.897959,NaN,NaN
3,00459419,Marko Arnautović,21.0,17.0,1530.0,18.0,10.0,7.0,4.8,7.2,...,0.282353,0.423529,0.882353,0.764706,2.117647,0.111765,70.235546,16.666667,NaN,NaN
4,004d185e,Tete Morente,32.0,23.0,2068.0,23.0,4.0,4.0,4.2,4.2,...,0.182609,0.182609,0.608696,2.608696,2.739130,0.047826,72.738854,41.666667,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2720,ff98492b,Robert Skov,23.0,15.0,1349.0,16.0,3.0,1.0,1.8,3.4,...,0.120000,0.226667,0.800000,2.866667,3.266667,0.120000,69.917582,38.461538,NaN,NaN
2721,ffacd3d5,Manu Sánchez,31.0,23.6,2124.0,22.0,0.0,0.0,0.8,0.8,...,0.033898,0.033898,0.127119,1.779661,4.237288,0.088983,80.069930,52.380952,NaN,NaN
2722,ffec9769,Islam Slimani,16.0,11.0,986.0,11.0,1.0,1.0,1.7,2.5,...,0.154545,0.227273,0.181818,1.000000,2.363636,0.109091,71.117166,0.000000,NaN,NaN
2723,ffed43e3,Jack Hendry,4.0,2.4,220.0,2.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.416667,3.750000,0.000000,86.131387,0.000000,NaN,NaN


In [46]:
final_df.drop(columns=[col for col in final_df.columns if '-' in col], inplace=True)
final_df.columns

Index(['Player ID', 'Player', 'Matches Played', '90s', 'Minutes Played',
       'Starts', 'Goals', 'npGoals', 'npxG', 'xG', 'PrgCarries', 'PrgPasses',
       'CS', 'SoTA', 'Saves', 'GA', 'Crosses Stp', 'Crosses Opp', 'SoT',
       'Passes Cmp', 'Passes Att', 'Ast', 'xAG', 'Passes Final 3rd',
       'Challenges Tkl', 'Challenges Against Att', 'Tkl+Int',
       'Tackles Def 3rd', 'Blocks', 'Clr', 'Goals_p90', 'npGoals_p90',
       'npxG_p90', 'xG_p90', 'SoT_p90', 'PrgCarries_p90', 'PrgPasses_p90',
       'xAG_p90', '%PassesCmp', '%TklCmp', '%Saves', '%CossesStopped'],
      dtype='object')

### Saving

In [47]:
filename = 'tabular_data.csv'
final_df.to_csv(osp.join('new_data', filename), index = False)

In [48]:
filename = 'tabular_data.csv'
final_df = pd.read_csv(osp.join('new_data',filename))

final_df

,Player ID,Player,Matches Played,90s,Minutes Played,Starts,Goals,npGoals,npxG,xG,...,npxG_p90,xG_p90,SoT_p90,PrgCarries_p90,PrgPasses_p90,xAG_p90,%PassesCmp,%TklCmp,%Saves,%CossesStopped
0,0000acda,Marco Benassi,17.0,12.1,1091.0,12.0,0.0,0.0,1.3,1.3,...,0.107438,0.107438,0.661157,0.826446,2.975207,0.041322,75.501114,64.285714,NaN,NaN
1,00242715,Moussa Niakhate,14.0,12.9,1163.0,14.0,0.0,0.0,0.6,0.6,...,0.046512,0.046512,0.232558,0.155039,1.162791,0.007752,67.774936,50.000000,NaN,NaN
2,00242b75,Leandro Barreiro Martins,31.0,23.8,2141.0,25.0,4.0,4.0,3.9,3.9,...,0.163866,0.163866,0.588235,0.714286,2.268908,0.092437,68.043088,44.897959,NaN,NaN
3,00459419,Marko Arnautović,21.0,17.0,1530.0,18.0,10.0,7.0,4.8,7.2,...,0.282353,0.423529,0.882353,0.764706,2.117647,0.111765,70.235546,16.666667,NaN,NaN
4,004d185e,Tete Morente,32.0,23.0,2068.0,23.0,4.0,4.0,4.2,4.2,...,0.182609,0.182609,0.608696,2.608696,2.739130,0.047826,72.738854,41.666667,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2720,ff98492b,Robert Skov,23.0,15.0,1349.0,16.0,3.0,1.0,1.8,3.4,...,0.120000,0.226667,0.800000,2.866667,3.266667,0.120000,69.917582,38.461538,NaN,NaN
2721,ffacd3d5,Manu Sánchez,31.0,23.6,2124.0,22.0,0.0,0.0,0.8,0.8,...,0.033898,0.033898,0.127119,1.779661,4.237288,0.088983,80.069930,52.380952,NaN,NaN
2722,ffec9769,Islam Slimani,16.0,11.0,986.0,11.0,1.0,1.0,1.7,2.5,...,0.154545,0.227273,0.181818,1.000000,2.363636,0.109091,71.117166,0.000000,NaN,NaN
2723,ffed43e3,Jack Hendry,4.0,2.4,220.0,2.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.416667,3.750000,0.000000,86.131387,0.000000,NaN,NaN


### Melt

In [49]:
final_df_melted = final_df.melt(id_vars=['Player ID', 'Player'], var_name='Metric')
final_df_melted['value'] = pd.to_numeric(final_df_melted['value'], errors='coerce')

In [50]:
filename = 'melt_data.csv'
final_df_melted.to_csv(osp.join('new_data',filename), index=False)

In [51]:
metrics_percentage

{'%PassesCmp': ['Passes Cmp', 'Passes Att'],
 '%TklCmp': ['Challenges Tkl', 'Challenges Against Att'],
 '%Saves': ['Saves', 'SoTA'],
 '%CossesStopped': ['Crosses Stp', 'Crosses Opp']}

In [52]:
filename = 'melt_data.csv'
final_df_melted = pd.read_csv(osp.join('new_data',filename))
final_df_melted[final_df_melted.Metric.isin(metrics_percentage.keys())]

,Player ID,Player,Metric,value
98100,0000acda,Marco Benassi,%PassesCmp,75.501114
98101,00242715,Moussa Niakhate,%PassesCmp,67.774936
98102,00242b75,Leandro Barreiro Martins,%PassesCmp,68.043088
98103,00459419,Marko Arnautović,%PassesCmp,70.235546
98104,004d185e,Tete Morente,%PassesCmp,72.738854
...,...,...,...,...
108995,ff98492b,Robert Skov,%CossesStopped,NaN
108996,ffacd3d5,Manu Sánchez,%CossesStopped,NaN
108997,ffec9769,Islam Slimani,%CossesStopped,NaN
108998,ffed43e3,Jack Hendry,%CossesStopped,NaN


### Grid

In [18]:
grid_df = final_df.copy()
grid_df = grid_df[:10]
grid_df[grid_df.columns] = np.arange(10,110,10)[:, np.newaxis].repeat(grid_df.shape[1]).reshape(grid_df.shape)

grid_df.loc[9, :] = grid_df.loc[9, :]+1
grid_df['Player'] = 'grid'
grid_df['Player ID'] = 'zzzzzzz'
grid_df = grid_df.melt(id_vars=['Player ID','Player'], var_name='Metric',value_name='grid_values')

grid_df

,Player ID,Player,Metric,grid_values
0,zzzzzzz,grid,Matches Played,10
1,zzzzzzz,grid,Matches Played,20
2,zzzzzzz,grid,Matches Played,30
3,zzzzzzz,grid,Matches Played,40
4,zzzzzzz,grid,Matches Played,50
...,...,...,...,...
415,zzzzzzz,grid,%CossesStopped,60
416,zzzzzzz,grid,%CossesStopped,70
417,zzzzzzz,grid,%CossesStopped,80
418,zzzzzzz,grid,%CossesStopped,90


In [19]:
filename = 'grid.csv'
grid_df.to_csv(osp.join('new_data', filename), index=False)

### Dataset di prova

In [59]:
anagrafical_df = pd.read_csv(osp.join('new_data', 'anagrafical_data.csv'))
anagrafical_df.rename(columns={'Player_id': 'Player ID'}, inplace=True)
anagrafical_df.head()

,Player,Nation,Pos,Squad,Comp,Age,Main_position,Second_position,Nation_comp,Player ID
0,Brenden Aaronson,USA,"MF,FW",Leeds United,Premier League,21,MF,FW,United Kingdom,5bc43860
1,Paxten Aaronson,USA,"MF,DF",Eint Frankfurt,Bundesliga,18,MF,DF,Germany,4cd41883
2,James Abankwah,IRL,DF,Udinese,Serie A,18,DF,NaN,Italy,4b7a9fcc
3,George Abbott,ENG,MF,Tottenham,Premier League,16,MF,NaN,United Kingdom,56628958
4,Yunis Abdelhamid,MAR,DF,Reims,Ligue 1,34,DF,NaN,France,32c2d95f


In [60]:
final_df = pd.read_csv(osp.join('new_data', 'tabular_data.csv'))
final_df.head()

,Player ID,Player,Matches Played,90s,Minutes Played,Starts,Goals,npGoals,npxG,xG,...,npxG_p90,xG_p90,SoT_p90,PrgCarries_p90,PrgPasses_p90,xAG_p90,%PassesCmp,%TklCmp,%Saves,%CossesStopped
0,0000acda,Marco Benassi,17.0,12.1,1091.0,12.0,0.0,0.0,1.3,1.3,...,0.107438,0.107438,0.661157,0.826446,2.975207,0.041322,75.501114,64.285714,NaN,NaN
1,00242715,Moussa Niakhate,14.0,12.9,1163.0,14.0,0.0,0.0,0.6,0.6,...,0.046512,0.046512,0.232558,0.155039,1.162791,0.007752,67.774936,50.000000,NaN,NaN
2,00242b75,Leandro Barreiro Martins,31.0,23.8,2141.0,25.0,4.0,4.0,3.9,3.9,...,0.163866,0.163866,0.588235,0.714286,2.268908,0.092437,68.043088,44.897959,NaN,NaN
3,00459419,Marko Arnautović,21.0,17.0,1530.0,18.0,10.0,7.0,4.8,7.2,...,0.282353,0.423529,0.882353,0.764706,2.117647,0.111765,70.235546,16.666667,NaN,NaN
4,004d185e,Tete Morente,32.0,23.0,2068.0,23.0,4.0,4.0,4.2,4.2,...,0.182609,0.182609,0.608696,2.608696,2.739130,0.047826,72.738854,41.666667,NaN,NaN


In [61]:
combined_df = anagrafical_df.merge(final_df, on=['Player ID', 'Player'])
combined_df.head()

,Player,Nation,Pos,Squad,Comp,Age,Main_position,Second_position,Nation_comp,Player ID,...,npxG_p90,xG_p90,SoT_p90,PrgCarries_p90,PrgPasses_p90,xAG_p90,%PassesCmp,%TklCmp,%Saves,%CossesStopped
0,Brenden Aaronson,USA,"MF,FW",Leeds United,Premier League,21,MF,FW,United Kingdom,5bc43860,...,0.147727,0.147727,0.340909,1.628788,3.257576,0.159091,74.278545,32.608696,NaN,NaN
1,Paxten Aaronson,USA,"MF,DF",Eint Frankfurt,Bundesliga,18,MF,DF,Germany,4cd41883,...,0.105263,0.105263,0.526316,4.210526,3.157895,0.000000,71.830986,66.666667,NaN,NaN
2,James Abankwah,IRL,DF,Udinese,Serie A,18,DF,NaN,Italy,4b7a9fcc,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,79.310345,0.000000,NaN,NaN
3,George Abbott,ENG,MF,Tottenham,Premier League,16,MF,NaN,United Kingdom,56628958,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,NaN,NaN
4,Yunis Abdelhamid,MAR,DF,Reims,Ligue 1,34,DF,NaN,France,32c2d95f,...,0.064865,0.064865,0.081081,1.081081,5.810811,0.027027,82.668636,75.000000,NaN,NaN


In [10]:
leagues = ['Serie A', 'Bundesliga']
leagues_df = combined_df.loc[combined_df.Comp.isin(leagues), :]
leagues_df

,Player,Nation,Pos,Squad,Comp,Age,Main_position,Second_position,Nation_comp,Player ID,...,npxG_p90,xG_p90,SoT_p90,PrgCarries_p90,PrgPasses_p90,xAG_p90,%PassesCmp,%TklCmp,%Saves,%CossesStopped
1,Paxten Aaronson,USA,"MF,DF",Eint Frankfurt,Bundesliga,18,MF,DF,Germany,4cd41883,...,0.105263,0.105263,0.526316,4.210526,3.157895,0.000000,0.718310,0.666667,NaN,NaN
2,James Abankwah,IRL,DF,Udinese,Serie A,18,DF,NaN,Italy,4b7a9fcc,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.793103,0.000000,NaN,NaN
8,Oliver Abildgaard,DEN,MF,Hellas Verona,Serie A,26,MF,NaN,Italy,8f5cdf66,...,0.015152,0.015152,0.303030,0.303030,2.272727,0.060606,0.589744,0.437500,NaN,NaN
12,Tammy Abraham,ENG,FW,Roma,Serie A,24,FW,NaN,Italy,f586779e,...,0.444444,0.444444,1.152263,1.440329,2.222222,0.148148,0.666667,0.187500,NaN,NaN
13,Christian Acella,ITA,MF,Cremonese,Serie A,20,MF,NaN,Italy,771a2cfe,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2713,Nadir Zortea,ITA,"DF,MF",Atalanta,Serie A,23,DF,MF,Italy,82598b71,...,0.030612,0.030612,0.510204,3.571429,4.795918,0.132653,0.728346,0.666667,NaN,NaN
2716,Petar Zovko,BIH,GK,Spezia,Serie A,20,GK,NaN,Italy,54f6afe3,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.611111,1.000000,0.333333,0.041667
2719,Szymon Żurkowski,POL,MF,Fiorentina,Serie A,24,MF,NaN,Italy,4e1d5e59,...,0.137255,0.137255,0.392157,2.352941,2.156863,0.000000,0.725000,0.133333,NaN,NaN
2721,Milan Đurić,BIH,FW,Hellas Verona,Serie A,32,FW,NaN,Italy,405f6586,...,0.092105,0.092105,0.328947,0.131579,1.907895,0.085526,0.512428,0.333333,NaN,NaN


In [12]:
leagues_df.columns

Index(['Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Main_position',
       'Second_position', 'Nation_comp', 'Player ID', 'Matches Played', '90s',
       'Minutes Played', 'Starts', 'Goals', 'npGoals', 'npxG', 'xG',
       'PrgCarries', 'PrgPasses', 'CS', 'SoTA', 'Saves', 'GA', 'PSxGA - GA',
       'Crosses Stp', 'Crosses Opp', 'SoT', 'Passes Cmp', 'Passes Att', 'Ast',
       'xAG', 'A-xAG', 'Passes Final 3rd', 'Challenges Tkl',
       'Challenges Against Att', 'Tkl+Int', 'Tackles Def 3rd', 'Blocks', 'Clr',
       'Goals_p90', 'npGoals_p90', 'npxG_p90', 'xG_p90', 'SoT_p90',
       'PrgCarries_p90', 'PrgPasses_p90', 'xAG_p90', '%PassesCmp', '%TklCmp',
       '%Saves', '%CossesStopped'],
      dtype='object')

In [14]:
minutes_played = 1500
minutes_played_df = leagues_df.loc[leagues_df['Minutes Played'] > minutes_played, :]
minutes_played_df

,Player,Nation,Pos,Squad,Comp,Age,Main_position,Second_position,Nation_comp,Player ID,...,npxG_p90,xG_p90,SoT_p90,PrgCarries_p90,PrgPasses_p90,xAG_p90,%PassesCmp,%TklCmp,%Saves,%CossesStopped
12,Tammy Abraham,ENG,FW,Roma,Serie A,24,FW,NaN,Italy,f586779e,...,0.444444,0.444444,1.152263,1.440329,2.222222,0.148148,0.666667,0.187500,NaN,NaN
14,Francesco Acerbi,ITA,DF,Inter,Serie A,34,DF,NaN,Italy,b96b595c,...,0.037175,0.037175,0.074349,1.449814,3.717472,0.040892,0.885036,0.740741,NaN,NaN
26,Michel Aebischer,SUI,"FW,MF",Bologna,Serie A,25,FW,MF,Italy,f9c927de,...,0.087719,0.087719,0.116959,0.818713,2.690058,0.035088,0.826409,0.111111,NaN,NaN
33,Kevin Agudelo,COL,"MF,FW",Spezia,Serie A,23,MF,FW,Italy,46fa7204,...,0.063291,0.063291,0.295359,3.417722,4.641350,0.126582,0.722449,0.423729,NaN,NaN
42,Emanuel Aiwum,AUT,DF,Cremonese,Serie A,21,DF,NaN,Italy,d810c13b,...,0.010870,0.010870,0.054348,1.032609,2.500000,0.021739,0.790698,0.702128,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,Andre-Frank Zambo Anguissa,CMR,MF,Napoli,Serie A,26,MF,NaN,Italy,6bb2c084,...,0.129794,0.129794,0.442478,1.710914,5.339233,0.076696,0.879257,0.250000,NaN,NaN
2688,Alessandro Zanoli,ITA,DF,Sampdoria,Serie A,21,DF,NaN,Italy,3b12b11f,...,0.156977,0.156977,0.465116,2.965116,2.093023,0.069767,0.765120,0.470588,NaN,NaN
2701,Robin Zentner,GER,GK,Mainz 05,Bundesliga,27,GK,NaN,Germany,5c4bf6ba,...,0.000000,0.000000,0.000000,0.000000,0.230769,0.003846,0.612903,0.000000,0.633028,0.133956
2706,Piotr Zieliński,POL,MF,Napoli,Serie A,28,MF,NaN,Italy,a5296e55,...,0.137795,0.137795,0.748031,2.440945,5.039370,0.291339,0.837629,0.480000,NaN,NaN


In [27]:
role_df = {}

for role in minutes_played_df.Main_position.unique():
    role_df[role] = minutes_played_df.loc[(minutes_played_df.Comp == 'Serie A') & (minutes_played_df.Main_position == role), :].sample(n=2)
    role_df[role] = pd.concat([role_df[role], minutes_played_df.loc[(minutes_played_df.Comp == 'Bundesliga')& (minutes_played_df.Main_position == role), :].sample(n=2)], ignore_index=True)

In [58]:
test_df = pd.concat(role_df.values(), ignore_index=True)

In [62]:
combined_df.groupby(by=['Comp'])[['xG', 'Goals']].mean()

,xG,Goals
Comp,,
Bundesliga,1.835565,1.924686
La Liga,1.746561,1.611993
Ligue 1,1.950180,1.839928
Premier League,2.077076,1.947653
Serie A,1.731930,1.661404


In [63]:
combined_df.groupby(by=['Comp', 'Main_position'])[['xG', 'Goals']].mean()

xG     Goals
Comp           Main_position                    
Bundesliga     DF             0.888202  0.842697
               FW             3.704959  3.917355
               GK             0.012121  0.000000
               MF             1.853425  2.027397
La Liga        DF             0.712821  0.635897
               FW             4.181560  3.985816
               GK             0.000000  0.000000
               MF             1.392021  1.212766
Ligue 1        DF             0.857843  0.632353
               FW             4.425397  4.539683
               GK             0.000000  0.000000
               MF             1.880749  1.721925
Premier League DF             0.844211  0.589474
               FW             4.363399  4.287582
               GK             0.005128  0.000000
               MF             1.875000  1.808140
Serie A        DF             0.934579  0.850467
               FW             4.205833  4.150000
               GK             0.000000  0.000000
               MF             1.502660  1.420213

In [6]:
combined_df.loc[combined_df['Squad'] == 'Manchester City', :].groupby(by=['Main_position'])[['xG', 'Goals']].mean()

,xG,Goals
Main_position,,
DF,0.820000,0.6
FW,10.300000,13.2
GK,0.000000,0.0
MF,3.057143,3.0


In [10]:
combined_df.loc[combined_df['Squad'] == 'Manchester City', :][['xG', 'Goals']].mean()

xG       3.379167
Goals    3.875000
dtype: float64

In [57]:
combined_df.loc[(combined_df['Squad'] == 'Manchester City') & (combined_df.Main_position.isin(['FW'])), :][['xG', 'Goals']].mean()

xG       10.3
Goals    13.2
dtype: float64

In [58]:
combined_df.loc[(combined_df.Main_position.isin(['FW']))  , ['Comp','xG', 'Goals']].groupby(by='Comp').mean()

,xG,Goals
Comp,,
Bundesliga,3.704959,3.917355
La Liga,4.181560,3.985816
Ligue 1,4.425397,4.539683
Premier League,4.363399,4.287582
Serie A,4.205833,4.150000


In [ ]:
combined_df.loc[(combined_df.Main_position.isin(['GK','DF']))]

In [50]:
test_df.groupby(by='Main_position')['xG'].mean()

Main_position
DF    1.00
FW    5.20
GK    0.00
MF    4.55
Name: xG, dtype: float64

In [65]:
combined_df.groupby(by=['Comp', 'Main_position'])['xG'].mean()

Comp            Main_position
Bundesliga      DF               0.888202
                FW               3.704959
                GK               0.012121
                MF               1.853425
La Liga         DF               0.712821
                FW               4.181560
                GK               0.000000
                MF               1.392021
Ligue 1         DF               0.857843
                FW               4.425397
                GK               0.000000
                MF               1.880749
Premier League  DF               0.844211
                FW               4.363399
                GK               0.005128
                MF               1.875000
Serie A         DF               0.934579
                FW               4.205833
                GK               0.000000
                MF               1.502660
Name: xG, dtype: float64

In [69]:
roles = ['FW','MF']
combined_df.loc[combined_df.Main_position.isin(roles),:].groupby(by=['Comp'])['xG'].mean()

Comp
Bundesliga        2.692509
La Liga           2.587538
Ligue 1           2.905112
Premier League    3.046462
Serie A           2.555844
Name: xG, dtype: float64

In [51]:
(5.5+4.15)/2

4.825

In [39]:
test_anagrafical_df = test_df[anagrafical_df.columns]
test_anagrafical_df.to_csv(osp.join('test', 'anagrafical_df.csv'), index=False)
test_anagrafical_df

,Player,Nation,Pos,Squad,Comp,Age,Main_position,Second_position,Nation_comp,Player ID
0,Christian Kouamé,CIV,FW,Fiorentina,Serie A,24,FW,NaN,Italy,58e933d2
1,Andrea Petagna,ITA,FW,Monza,Serie A,27,FW,NaN,Italy,03f946ed
2,Sheraldo Becker,SUR,FW,Union Berlin,Bundesliga,27,FW,NaN,Germany,8eb55dbb
3,Lucas Höler,GER,"FW,MF",Freiburg,Bundesliga,28,FW,MF,Germany,ca618d23
4,Leonardo Spinazzola,ITA,DF,Roma,Serie A,29,DF,NaN,Italy,3626daea
5,Koray Günter,GER,DF,Hellas Verona,Serie A,27,DF,NaN,Italy,f1f1f238
6,Kiliann Sildillia,FRA,"DF,MF",Freiburg,Bundesliga,20,DF,MF,Germany,313254a3
7,Benjamin Henrichs,GER,"DF,MF",RB Leipzig,Bundesliga,25,DF,MF,Germany,0a89e877
8,Lewis Ferguson,SCO,"MF,FW",Bologna,Serie A,22,MF,FW,Italy,ee64a822
9,Darko Lazović,SRB,"MF,DF",Hellas Verona,Serie A,31,MF,DF,Italy,5128bde4


In [53]:
test_df = test_df[final_df.columns]
test_df.to_csv(osp.join('test', 'tabular_data.csv'), index=False)
test_df

,Player ID,Player,Matches Played,90s,Minutes Played,Starts,Goals,npGoals,npxG,xG,...,npxG_p90,xG_p90,SoT_p90,PrgCarries_p90,PrgPasses_p90,xAG_p90,%PassesCmp,%TklCmp,%Saves,%CossesStopped
0,58e933d2,Christian Kouamé,28.0,20.7,1866.0,22.0,4.0,4.0,5.3,5.3,...,0.256039,0.256039,0.628019,2.415459,2.608696,0.178744,0.708018,0.500000,NaN,NaN
1,03f946ed,Andrea Petagna,31.0,18.5,1663.0,19.0,4.0,3.0,4.9,5.7,...,0.264865,0.308108,0.648649,1.243243,1.837838,0.237838,0.728111,0.500000,NaN,NaN
2,8eb55dbb,Sheraldo Becker,34.0,28.4,2554.0,33.0,11.0,11.0,6.5,6.5,...,0.228873,0.228873,0.915493,3.697183,1.549296,0.197183,0.593558,0.181818,NaN,NaN
3,ca618d23,Lucas Höler,26.0,16.7,1503.0,16.0,5.0,5.0,3.3,3.3,...,0.197605,0.197605,0.898204,1.137725,1.976048,0.113772,0.650104,0.272727,NaN,NaN
4,3626daea,Leonardo Spinazzola,26.0,18.4,1654.0,18.0,1.0,1.0,1.0,1.0,...,0.054348,0.054348,0.326087,5.815217,3.423913,0.108696,0.758342,0.347826,NaN,NaN
5,f1f1f238,Koray Günter,23.0,19.7,1779.0,19.0,1.0,1.0,1.0,1.0,...,0.050761,0.050761,0.050761,0.558376,2.893401,0.055838,0.792162,0.625000,NaN,NaN
6,313254a3,Kiliann Sildillia,27.0,22.0,1977.0,23.0,0.0,0.0,0.8,0.8,...,0.036364,0.036364,0.000000,0.909091,3.681818,0.031818,0.713509,0.600000,NaN,NaN
7,0a89e877,Benjamin Henrichs,30.0,24.3,2188.0,23.0,2.0,2.0,1.2,1.2,...,0.049383,0.049383,0.164609,1.975309,4.485597,0.074074,0.778934,0.651515,NaN,NaN
8,ee64a822,Lewis Ferguson,32.0,25.7,2314.0,27.0,7.0,7.0,4.7,4.7,...,0.182879,0.182879,0.700389,1.206226,3.424125,0.038911,0.853038,0.500000,NaN,NaN
9,5128bde4,Darko Lazović,30.0,24.9,2242.0,27.0,4.0,4.0,3.6,3.6,...,0.144578,0.144578,0.562249,2.891566,3.253012,0.257028,0.618421,0.448276,NaN,NaN


In [56]:
test_df.melt(id_vars=['Player ID', 'Player']).to_csv(osp.join('test', 'melted_data.csv'), index=False)

### Average Performances